In [48]:
import pandas as pd
import warnings
import glob
from os import path
warnings.filterwarnings("ignore")

In [64]:
def get_files_name(path_to_files):
    names = []
    for file in glob.glob(path_to_files):
        names.append(path.splitext(path.basename(file))[0])
    return names

def load_data(path):
    df = pd.read_csv(path, compression= "zip")
    df_searched = df.query("search_conducted == True")
    df_searched["date"] = pd.to_datetime(df_searched["date"])
    df_searched["year"] = df_searched["date"].dt.year
    df_searched = df_searched.query('year >= 2009 and year <= 2016')
    #df_searched['county_name'] = df_searched['county_name'].str[:-7]
    return df_searched

def compute_hit_rate_one_object(df, county, city, object = 'ethnicity'):    
    if object == 'ethnicity':
        df_searched_cleaned = df[['year', 'subject_race', 'search_conducted', 'contraband_found']]
        df_searched_cleaned = df_searched_cleaned.query("subject_race == 'white' | subject_race == 'black'| subject_race == 'asian/pacific islander'\
            | subject_race == 'hispanic'")
        found = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_race'])['contraband_found'].value_counts()).rename(columns={'contraband_found' : 'nb_find'}).reset_index()
        all = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_race'])['search_conducted'].value_counts()).rename(columns={'search_conducted' : 'nb_search'}).reset_index()
        found_and_all = pd.merge(left = found, right = all, left_on=['year','subject_race'], right_on = ['year', 'subject_race']).drop(['search_conducted'], axis = 1)
        found_and_all['Hit_rate'] = found_and_all['nb_find'] / found_and_all['nb_search'] * 100
        found_and_all['County'] = county
        found_and_all['City'] = city
    else :
        df_searched_cleaned = df[['year', 'subject_sex', 'search_conducted', 'contraband_found']]
        found = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_sex'])['contraband_found'].value_counts()).rename(columns={'contraband_found' : 'nb_find'}).reset_index()
        all = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_sex'])['search_conducted'].value_counts()).rename(columns={'search_conducted' : 'nb_search'}).reset_index()
        found_and_all = pd.merge(left = found, right = all, left_on=['year','subject_sex'], right_on = ['year', 'subject_sex']).drop(['search_conducted'], axis = 1)
        found_and_all['Hit_rate'] = found_and_all['nb_find'] / found_and_all['nb_search'] * 100
        found_and_all['County'] = county
        found_and_all['City'] = city
        
    return found_and_all.query('contraband_found == True')

def compute_hit_rate_one_city(path,county, city):
    df = load_data('../data/city/'+path +'.zip')
    df_ethnicity = compute_hit_rate_one_object(df,county,city)
    df_gender = compute_hit_rate_one_object(df,county,city, object = 'gender')
    return df_ethnicity, df_gender

def compute_hit_rate_multiple_cities(path_to_folder):
    file_names = get_files_name(path_to_folder)
    hit_rate_ethnicity, hit_rate_gender = compute_hit_rate_one_city(file_names[0], file_names[0][0:2], file_names[0][3:-4]) 
    for i in range(1,len(file_names)):
        new_city_ethnicity, new_city_gender = compute_hit_rate_one_city(file_names[i],file_names[i][0:2] , file_names[0][3:-4])
        hit_rate_ethnicity = pd.concat([hit_rate_ethnicity, new_city_ethnicity])
        hit_rate_gender = pd.concat([hit_rate_gender, new_city_gender])
    return hit_rate_ethnicity, hit_rate_gender

def filter_not_duplicate(df, object = 'ethni'):
    if object == 'ethni':
        dupli = pd.DataFrame(df.groupby(['year'])['subject_race'].value_counts()).rename(columns={'subject_race' : 'dupli'}).reset_index()
        test = pd.merge(left=df, right=dupli, left_on = ['year','subject_race'], right_on = ['year', 'subject_race'])
        df = test.query('dupli == 2')
    else :
        dupli = pd.DataFrame(df.groupby(['year'])['subject_sex'].value_counts()).rename(columns={'subject_sex' : 'dupli'}).reset_index()
        test = pd.merge(left=df, right=dupli, left_on = ['year','subject_sex'], right_on = ['year', 'subject_sex'])
        test = test.query('dupli == 2')
    return df

def compute_mean_hit_rate_by_state(path_to_folder):
    hit_rate_ethnicity, hit_rate_gender = compute_hit_rate_multiple_cities(path_to_folder)
    all_city_gender_mean_hit_rate = hit_rate_gender.groupby(['year', 'subject_sex', 'County'])['Hit_rate'].agg(['mean']).reset_index()
    all_city_gender_mean_search_count = hit_rate_gender.groupby(['year', 'subject_sex'])['nb_search'].agg(['sum']).reset_index()
    hit_rate_gender_mean = pd.merge(all_city_gender_mean_hit_rate, all_city_gender_mean_search_count, left_on=['year', 'subject_sex'], right_on=['year', 'subject_sex'])
    all_city_ethnicity_mean_hit_rate = hit_rate_ethnicity.groupby(['year', 'subject_race', 'County'])['Hit_rate'].agg(['mean']).reset_index()
    all_city_ethnicity_mean_search_count = hit_rate_ethnicity.groupby(['year', 'subject_race'])['nb_search'].agg(['sum']).reset_index()
    hit_rate_ethnicity_mean = pd.merge(all_city_ethnicity_mean_hit_rate, all_city_ethnicity_mean_search_count, left_on=['year', 'subject_race'], right_on=['year', 'subject_race'])
    return filter_not_duplicate(hit_rate_ethnicity_mean), filter_not_duplicate(hit_rate_gender_mean, object = 'gender')\
        , filter_not_duplicate(hit_rate_ethnicity), filter_not_duplicate(hit_rate_gender, object = 'gender')


In [65]:
hit_rate_ethnicity_mean, hit_rate_gender_mean, hit_rate_ethnicity, hit_rate_gender = compute_mean_hit_rate_by_state('../data/city/*.csv.zip')

In [70]:
hit_rate_ethnicity.to_csv('../data/city/all_hit_rate_ethnicity.csv')
hit_rate_gender.to_csv('../data/city/all_hit_rate_gender.csv')
hit_rate_ethnicity_mean.to_csv('../data/city/all_hit_rate_ethnicity_mean.csv')
hit_rate_gender_mean.to_csv('../data/city/all_hit_rate_gender_mean.csv')